In [1]:
#list Converter to dictionary List
def listCoverter():
    count=0
    worddict={}
    with open("NRC-Emotion-Lexicon-Wordlevel-v0.92.txt","r") as file:
        for line in file:
            if count<1:
                count+=1
                continue
            line=line.strip().split("\t")
            if int(line[2])==1:
        # this entire operations loops through a list and create a dictionary list out of it.
               if  worddict.get(line[0]):
                   worddict[line[0]].append(line[1])
               else:
                   worddict[line[0]]=[line[1]]
    return worddict

In [2]:
EmotionDict=listCoverter()
EmotionDict['fear']

['anger', 'fear', 'negative']

In [3]:
# Connecting this Emotion diction with Yelp API reviews
import pandas as pd
import requests


def YelpApi(url,):

    # Define my endpoints
    # define my Api key
    api_key = "4onc8AiIdwpXPD77I6QlHAbmKe8DZfFzK-cz6sglyW1Tnp0ng8QKWfRVCwS6Z2qV4aXE5FnqxfGZD9y2s7gCUOsgx5T9Xp9iY7dFhDEYq7fGEXYGsLt-bSN0JVWSXnYx"
    # Alternative connection header = {"Authorization": "Bearer %s" % api_key}
    header = {"Authorization": "Bearer " + api_key}

    # Define the terms of search
    parameters = {
                  'term': 'Coffee',
                  'limit': 50,
                  'radius': 1000,
                  'location': 'Houston'
                  }
    # maing my request to yelp
    response = requests.get(url=url, headers=header, params=parameters)

    # json() convert the returning data into a dictionary
    business_data = response.json()
    print(business_data)
    #  index into the dictionay and the list as shown below
    for businesslist in business_data["businesses"][0:]:
        for listbreak in businesslist.items():
            print(listbreak)

    """businesbreakdown = business_data["businesses"][0:]

    # Loop through the dictionary to key the keys
    for key in businesbreakdown.items():  # to return key and values
        print(key)
    return key


# to covert to DataFrame for analysis
df = pd.DataFrame.from_dict(businesbreakdown, orient='index')
print(df.transpose()["phone"])"""

#  You can call this from another program as a module as shown below.
url = "https://api.yelp.com/v3/businesses/search"
YelpApi(url)



{'businesses': [{'id': '_vkxrtWVgt5H2RUp1RZZXA', 'alias': 'common-grounds-uncommon-coffee-houston-3', 'name': 'Common Grounds Uncommon Coffee', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/BQLDTEGIkG98mSI-nsrtQQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/common-grounds-uncommon-coffee-houston-3?adjust_creative=f5AUaRnrKmGMZumVvYKtzA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=f5AUaRnrKmGMZumVvYKtzA', 'review_count': 18, 'categories': [{'alias': 'cafes', 'title': 'Cafes'}, {'alias': 'coffee', 'title': 'Coffee & Tea'}], 'rating': 5.0, 'coordinates': {'latitude': 29.75498, 'longitude': -95.36075}, 'transactions': [], 'price': '$', 'location': {'address1': '1401 McKinney St', 'address2': '', 'address3': None, 'city': 'Houston', 'zip_code': '77010', 'country': 'US', 'state': 'TX', 'display_address': ['1401 McKinney St', 'Houston, TX 77010']}, 'phone': '+17132992935', 'display_phone': '(713) 299-2935', 'distance': 874.7200657606055}, {'id'

In [4]:
#Buidling another API to return the reviews
# Connecting this Emotion diction with Yelp API reviews
import pandas as pd
import requests

bizlistReview=[]
def YelpApi(url,World_location):

    # Define my endpoints
    # define my Api key
    api_key = "4onc8AiIdwpXPD77I6QlHAbmKe8DZfFzK-cz6sglyW1Tnp0ng8QKWfRVCwS6Z2qV4aXE5FnqxfGZD9y2s7gCUOsgx5T9Xp9iY7dFhDEYq7fGEXYGsLt-bSN0JVWSXnYx"
    # Alternative connection header = {"Authorization": "Bearer %s" % api_key}
    header = {"Authorization": "Bearer " + api_key}

    # Define the terms of search
    parameters = {'locale':World_location
                  }
    # maing my request to yelp
    response = requests.get(url=url, headers=header, params=parameters)

    # json() convert the returning data into a dictionary
    business_data = response.json()
    #  index into the dictionay and the list as shown below
    for business in business_data["reviews"]:
       text=business["text"]
       bizid=business["id"]
       bizname=business["user"]["name"]
       bizlistReview.append((bizid,bizname,text))
    return bizlistReview
    

"""
# to covert to DataFrame for analysis
df = pd.DataFrame.from_dict(businesbreakdown, orient='index')
print(df.transpose()["phone"])"""

#  You can call this from another program as a module as shown below.
url = "https://api.yelp.com/v3/businesses/nhmfHKtIzTdhX8suTvyNBw/reviews"

YelpApi(url,"en_HK")

[('wkluZRYTfOf5HDuEfD1R0w',
  'Anita R.',
  'Went here for brunch and loved the concept of this place that is in a church complex/ loft style setting. This place is so unique in the sense that everyone...'),
 ('AxsPCt3c0f80l33DpxwHfQ',
  'Claire L.',
  "Always wanted to visit this cafe but always couldn't since I cannot make it on time before it closes... i finally was able to when I was around the area...."),
 ('QTEGv9G9k849zzOcBATiwA',
  'Joseph V.',
  'So we stumbled upon this place on Yelp and would have never known it was here if not. It is interesting because it is actually attached to a church! \n\nThe...')]

In [7]:

# This function takes the reviews and analyze it the emotion dictionary.
def emotion_analyzer(text,emotion_dic=EmotionDict):
# The operation below form a set operation on adictionary
    emotions={x for y in emotion_dic.values() for x in y}
    emotions_counts=dict()
    for emotion in emotions:
        emotions_counts[emotion]=0

    total_words=len(text.split())
    for word in text.split():
        if emotion_dic.get(word):
            for emotion in emotion_dic.get(word):
                emotions_counts[emotion]+=1/total_words
    return emotions_counts

In [8]:
print("%-12s %1s\t %1s %1s %1s %1s %1s %1s %1s %1s"%("restuarant","fear","trust","nagative","positive","joy","disgust","anticipation","sadness","surprise"))
for snippet in YelpApi(url,"en_HK"):
    text=snippet[2]
    result= emotion_analyzer(text)
    print("%-12s %1.2f\t %1.2f\t %1.2f\t %1.2f\t %1.2f\t %1.2f\t %1.2f\t %1.2f\t %1.2f\t"%(snippet[1][0:10],result["fear"],result["trust"],result["negative"],result["positive"],result["joy"],result["disgust"],result["anticipation"],
         result["sadness"],result["surprise"]))

restuarant   fear	 trust nagative positive joy disgust anticipation sadness surprise
Anita R.     0.00	 0.03	 0.00	 0.10	 0.03	 0.00	 0.03	 0.00	 0.03	
Claire L.    0.00	 0.03	 0.00	 0.10	 0.03	 0.03	 0.07	 0.00	 0.03	
Michelle W   0.00	 0.07	 0.00	 0.11	 0.04	 0.00	 0.04	 0.00	 0.00	
Anita R.     0.00	 0.03	 0.00	 0.10	 0.03	 0.00	 0.03	 0.00	 0.03	
Claire L.    0.00	 0.03	 0.00	 0.10	 0.03	 0.03	 0.07	 0.00	 0.03	
Michelle W   0.00	 0.07	 0.00	 0.11	 0.04	 0.00	 0.04	 0.00	 0.00	
